# Walkthrough

First step: Run the combin_csv.py script on a data folder containing ONLY the kickstarter000 to kickstarter055 csv-files! This gives you the dataset to import for this notebook.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import classification_report, confusion_matrix

In [3]:
# load the combined dataset
df = pd.read_csv("data/combined_csv.csv")

# check the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209222 entries, 0 to 209221
Data columns (total 37 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   backers_count             209222 non-null  int64  
 1   blurb                     209214 non-null  object 
 2   category                  209222 non-null  object 
 3   converted_pledged_amount  209222 non-null  int64  
 4   country                   209222 non-null  object 
 5   created_at                209222 non-null  int64  
 6   creator                   209222 non-null  object 
 7   currency                  209222 non-null  object 
 8   currency_symbol           209222 non-null  object 
 9   currency_trailing_code    209222 non-null  bool   
 10  current_currency          209222 non-null  object 
 11  deadline                  209222 non-null  int64  
 12  disable_communication     209222 non-null  bool   
 13  friends                   300 non-null     o

## list of features we keep

 -  blurb                     short description 
 -  category                  Kickstarter categories  
 -  country                   country 
 -  deadline                  deadline date/time?
 -  fx_rate                   currency conversion rate 
 -  goal                      fixed amount required for funding (convert with fx_rate)
 -  launched_at               launch date/time?  
 -  location                  location
 -  name                      project name 
 -  state                     !!!!!target!!!!! 

 <br>

features to create:
- goal_usd                    goal*fx_rate
- name_len                    number of characters in name
- blurb_len                   number of characters in blurb
- time_online                 deadline - launched at
- launch_weekday              day of the week of the launch
- launch_time                 time of day of the launch

## stakeholder

* who? - people/creators who are considering launching a project on Kickstarter
* why? - to find out if it's worth investing the time/money in creating materials/launching a project and which criteria to consider in order to make it successful
* metric? - f_beta (probably imbalanced data)
* model: classifier (binary)

In [4]:
# keep only columns we will be using

df = df[['blurb', 'category', 'country', 'deadline',
        'fx_rate', 'goal', 'launched_at', 'location',
       'name', 'state']]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209222 entries, 0 to 209221
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   blurb        209214 non-null  object 
 1   category     209222 non-null  object 
 2   country      209222 non-null  object 
 3   deadline     209222 non-null  int64  
 4   fx_rate      209222 non-null  float64
 5   goal         209222 non-null  float64
 6   launched_at  209222 non-null  int64  
 7   location     208996 non-null  object 
 8   name         209222 non-null  object 
 9   state        209222 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 16.0+ MB


## Create the new features

### make category column usable

In [5]:
# subset category column 
cat_df = df.loc[:,['category']]

# split category column
cat_df[['id','name','slug','position','parent_id','color','urls']] = df['category'].str.split(',',expand=True)

In [6]:
# check number of unique values
cat_df['slug'].nunique()

169

Problem:
- most rows have the format 'main category/ subcategory', but some do not have a subcategory
- this means splitting by '/' is difficult
- if we don't split any further, we have 169 unique values for category, seems okay

In [7]:
# subset column 'slug' as dataframe
cat = cat_df.loc[:,['slug']]

# rename 'slug' to 'category'
cat.rename(columns={'slug': 'category'}, inplace=True)

# drop column 'category' from original dataframe
df.drop('category', axis=1, inplace=True)

# add new column 'category' to original dataframe
df['category'] = cat
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209222 entries, 0 to 209221
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   blurb        209214 non-null  object 
 1   country      209222 non-null  object 
 2   deadline     209222 non-null  int64  
 3   fx_rate      209222 non-null  float64
 4   goal         209222 non-null  float64
 5   launched_at  209222 non-null  int64  
 6   location     208996 non-null  object 
 7   name         209222 non-null  object 
 8   state        209222 non-null  object 
 9   category     209222 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 16.0+ MB


### Convert timestamps & add time delta features

In [8]:
launched = []
deadline = []

In [9]:
# launched_at
for label, content in df['launched_at'].iteritems():
    launched.append(datetime.datetime.fromtimestamp(content))

In [10]:
# deadline
for label, content in df['deadline'].iteritems():
    deadline.append(datetime.datetime.fromtimestamp(content))

In [11]:
for i in range(len(df)):
    df['launched_at'].iloc[[i]] = launched[i]

/Users/zoe/neuefische/kickstarter-ml/.venv/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [12]:
for i in range(len(df)):
    df['deadline'].iloc[[i]] = deadline[i]

In [13]:
df['delta_dead_laun'] = (df['deadline'] - df['launched_at']).astype('timedelta64[h]')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209222 entries, 0 to 209221
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   blurb            209214 non-null  object 
 1   country          209222 non-null  object 
 2   deadline         209222 non-null  object 
 3   fx_rate          209222 non-null  float64
 4   goal             209222 non-null  float64
 5   launched_at      209222 non-null  object 
 6   location         208996 non-null  object 
 7   name             209222 non-null  object 
 8   state            209222 non-null  object 
 9   category         209222 non-null  object 
 10  delta_dead_laun  209222 non-null  float64
dtypes: float64(3), object(8)
memory usage: 17.6+ MB


### Include only projects that were successful or failed

In [ ]:
# convert 'state' to numerical
# successful: 1
# failed: 0
# drop: live, suspended, canceled

df = df.query('state != "live"')
df = df.query('state != "suspended"')
df = df.query('state != "canceled"')
print(df['state'].unique()) # check that 'state' only contains failed and successful

df['state'].replace({'failed':0, 'successful':1}, inplace=True)
print(df['state'].unique()) # check that 'state' only contains 1 and 0

In [ ]:
# plot frequency of success and failure

sns.countplot(x='state', data=df)